<a href="https://colab.research.google.com/github/dan-candeira/Nuforc-Project/blob/master/5.4-Exploracao/5.4-nuforc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
#! pip install pandasql
import pandasql as ps
import requests
from bs4 import BeautifulSoup 

In [9]:
df = pd.read_csv(
    'https://raw.githubusercontent.com/dan-candeira/Nuforc-Project/master/5.2-Coleta/ufoCollection.csv'
    ).drop(columns=['Unnamed: 0'])

,Date / Time,City,State,Shape,Duration,Summary,Posted
0,9/30/97 22:00,Madison,WI,Light,5 minutes,Strange light inside Lake Monona,3/2/04
1,9/30/97 20:00,Nova Scotia (Canada),NS,Light,8-10 seconds.,"Single light resembling a star, but moving spu...",10/30/06
2,9/28/97 23:15,San Francisco,CA,Triangle,12-15s,flying-wing shape outlined by 12-14 lights. Ap...,7/5/99
3,9/27/97 23:00,Egan,SD,Other,30 minutes,The Weirdest Thing I Have Ever Seen,2/22/05
4,9/27/97 05:00,Crestwood,KY,Disk,15 minutes,A big disk with red and green lights on the ri...,8/5/01
...,...,...,...,...,...,...,...
99700,8/1/17 14:00,Joliet,IL,Other,2 minutes,The White Cube UFO,7/25/19
99701,8/1/17 06:15,Columbus (North),GA,Fireball,3 seconds,Green streak growing in size moving from west ...,8/4/17
99702,8/1/17 02:45,Corcoran,MN,Light,Still going,Small light south west of Minneapolis maneuver...,8/4/17
99703,8/1/17 02:00,Moreno Valley,CA,Other,10 seconds,I was looking out the front windshield and loo...,8/4/17


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99705 entries, 0 to 99704
Data columns (total 7 columns):
Date / Time    99705 non-null object
City           99511 non-null object
State          92524 non-null object
Shape          97703 non-null object
Duration       96725 non-null object
Summary        99684 non-null object
Posted         99703 non-null object
dtypes: object(7)
memory usage: 5.3+ MB


In [5]:
df.isna().sum()

Date / Time       0
City            194
State          7181
Shape          2002
Duration       2980
Summary          21
Posted            2
dtype: int64

In [6]:
df.shape

(99705, 7)

#### Reports descending order by State

In [12]:
query = '''
  SELECT State, COUNT(Posted) as Reports 
  FROM df
  GROUP BY State 
  ORDER BY Reports DESC 
'''

per_state = ps.sqldf(query, locals())

,State,Reports
0,CA,11402
1,None,7181
2,FL,5577
3,WA,4901
4,TX,4154
...,...,...
64,PE,20
65,YT,19
66,PR,18
67,YK,5


#### Reports descending order by State without blank states

In [16]:
query = '''
  SELECT State, COUNT(Posted) as Reports FROM df
  WHERE State != ''
  GROUP BY State 
  ORDER BY Reports DESC 
'''

per_state_no_blanck = ps.sqldf(query, locals())

,State,Reports
0,CA,11402
1,FL,5577
2,WA,4901
3,TX,4154
4,NY,3871
...,...,...
63,PE,20
64,YT,19
65,PR,18
66,YK,5


#### Getting the list of USA - States

In [17]:
request = requests.get('https://simple.wikipedia.org/wiki/List_of_U.S._states').text
soup = BeautifulSoup(request, 'html.parser')
table = soup.find_all('table', class_='wikitable')
data = pd.read_html(str(table))[0]
data.head()

Name postal abbreviation[1]  ... Water area[4] Numberof Reps.
         Name postal abbreviation[1]  ...           km2 Numberof Reps.
0     Alabama                     AL  ...          4597              7
1      Alaska                     AK  ...        245384              1
2     Arizona                     AZ  ...          1026              9
3    Arkansas                     AR  ...          2961              4
4  California                     CA  ...         20501             53

[5 rows x 13 columns]

#### Getting abreviations from the data collected

In [20]:
states = data.loc[:, ['postal abbreviation[1]', 'Name']]
states.columns = ['State_Name', 'State_Abbr']

,State_Name,State_Abbr
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA
5,Colorado,CO
6,Connecticut,CT
7,Delaware,DE
8,Florida,FL
9,Georgia,GA


In [19]:
query = '''
  SELECT df.State, COUNT(df.Posted) as Reports FROM df, states
  WHERE State = State_Abbr
  GROUP BY State
'''
eua_posts = ps.sqldf(query, locals())
eua_posts.head()

,State,Reports
0,AK,449
1,AL,893
2,AR,761
3,AZ,3464
4,CA,11402


In [21]:
eua_posts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 2 columns):
State      50 non-null object
Reports    50 non-null int64
dtypes: int64(1), object(1)
memory usage: 928.0+ bytes


In [22]:
eua_posts.describe()

,Reports
count,50.000000
mean,1762.800000
std,1865.875764
min,154.000000
25%,752.750000
50%,1166.000000
75%,2049.750000
max,11402.000000


In [24]:
query = '''
  SELECT df.* FROM df, states
  WHERE State = State_Abbr
'''
eua_df = ps.sqldf(query, locals())

,Date / Time,City,State,Shape,Duration,Summary,Posted
0,9/30/97 22:00,Madison,WI,Light,5 minutes,Strange light inside Lake Monona,3/2/04
1,9/28/97 23:15,San Francisco,CA,Triangle,12-15s,flying-wing shape outlined by 12-14 lights. Ap...,7/5/99
2,9/27/97 23:00,Egan,SD,Other,30 minutes,The Weirdest Thing I Have Ever Seen,2/22/05
3,9/27/97 05:00,Crestwood,KY,Disk,15 minutes,A big disk with red and green lights on the ri...,8/5/01
4,9/25/97 22:00,Clearfield,UT,Triangle,60-90 seconds,We observed a low flying craft (aprox.100yards...,1/28/99
...,...,...,...,...,...,...,...
88137,8/1/17 14:00,Joliet,IL,Other,2 minutes,The White Cube UFO,7/25/19
88138,8/1/17 06:15,Columbus (North),GA,Fireball,3 seconds,Green streak growing in size moving from west ...,8/4/17
88139,8/1/17 02:45,Corcoran,MN,Light,Still going,Small light south west of Minneapolis maneuver...,8/4/17
88140,8/1/17 02:00,Moreno Valley,CA,Other,10 seconds,I was looking out the front windshield and loo...,8/4/17


In [26]:
query = '''  
  SELECT *, COUNT(Posted) as Reports FROM eua_df
  GROUP BY City
  ORDER BY Reports
'''

reports_by_city = ps.sqldf(query, locals())

,Date / Time,City,State,Shape,Duration,Summary,Posted,Reports
0,1/20/10 19:30,((HOAX??)),PA,Circle,22,"((HOAX?? Source indicates state of PA, in Ecua...",2/14/10,1
1,5/10/17 21:00,((Location no revealed by witness)),CA,Light,30 seconds,I saw something strange in the night sky disap...,5/11/17,1
2,10/23/08 04:45,((Location unspecified; rural area)),WY,Flash,0.001sec,"brilliant strobe light at 4am, moving lights, ...",1/10/09,1
3,6/30/13 14:00,((Unknown)),NJ,Circle,1 minute,"Grey circle (it was hard to notice, looked lik...",8/11/17,1
4,9/28/10 19:30,((Unspecified location)),IL,Chevron,1 second,Strobing Chevron,11/21/10,1
...,...,...,...,...,...,...,...,...
16595,8/31/17 21:00,San Diego,CA,Rectangle,30 seconds,Rectangle four white lights two red flashing l...,9/5/17,394
16596,8/14/17 14:00,Las Vegas,NV,Rectangle,15 minutes,A large silver rectangle shaped object in top ...,8/17/17,473
16597,8/4/17 21:40,Portland,OR,Circle,20 minutes,"7 very slowly moving glowing aircraft, close t...",8/11/17,480
16598,8/15/17 11:00,Seattle,WA,Changing,50 seconds,North Seattle late morning Orb pulsing differe...,8/24/17,548


In [30]:
query = '''
  SELECT City, Shape, Reports, State FROM reports_by_city, states
  WHERE Reports >= 10
  AND City != ''
  AND City != 'unknown'
  GROUP BY City
  ORDER BY Reports DESC
'''
city_reports = ps.sqldf(query, locals())

,City,Shape,Reports,State
0,Phoenix,Flash,558,AZ
1,Seattle,Changing,548,WA
2,Portland,Circle,480,OR
3,Las Vegas,Rectangle,473,NV
4,San Diego,Rectangle,394,CA
...,...,...,...,...
1833,Winslow,Changing,10,AZ
1834,Wolf Point,Flash,10,MT
1835,Woodville,Circle,10,TX
1836,Yarmouth,Teardrop,10,MA


In [33]:
query = '''
  SELECT *, COUNT(Posted) as State_reports FROM eua_df 
  GROUP BY State
'''
state_reports = ps.sqldf(query, locals())

,Date / Time,City,State,Shape,Duration,Summary,Posted,State_reports
0,7/27/17 05:55,Anchorage,AK,Diamond,2 seconds,At approximately 5:50 am I saw 4 bright lights...,7/27/17,449
1,8/8/17 20:30,Huntsville,AL,Disk,1 minute,Disc shape 3 green lights with odd movement.,8/11/17,893
2,8/18/17 11:00,Conway,AR,Cylinder,7-8 minutes,Object appeared to be cylinder and cigar-shape...,9/5/17,761
3,8/4/17 21:15,Phoenix,AZ,Flash,5 minutes,Streak of light ending with 3 objects flashing...,8/11/17,3464
4,8/1/17 02:00,Moreno Valley,CA,Other,10 seconds,I was looking out the front windshield and loo...,8/4/17,11402
5,8/9/17 13:15,Rabbit Ears Pass/Steamboat springs,CO,Triangle,5 seconds,A dark grey triangle over Rabbit Ear's Pass.,8/17/17,2106
6,8/5/17 17:00,Norwalk/Darien,CT,Sphere,45 seconds,My friend and I where traveling on i95 South. ...,9/12/17,1192
7,8/14/17 17:20,Selbyville,DE,Rectangle,5 seconds,"Bright light in the sky over the trees, a rect...",9/5/17,271
8,8/1/17 01:00,Bradenton,FL,Other,<20 seconds,"I was walking my dog about 1am on August 1, 20...",5/9/19,5577
9,8/1/17 06:15,Columbus (North),GA,Fireball,3 seconds,Green streak growing in size moving from west ...,8/4/17,1802


Enfatizar a cidade, a quantidade de relatos e formato do objeto não identificado

In [35]:
query = '''
  SELECT State, MAX(Reports) as State_reports  FROM per_state
'''
most_reported_state = ps.sqldf(query, locals())

,State,State_reports
0,CA,11402


In [37]:
query = '''
  SELECT city_reports.City, city_reports.Shape, city_reports.Reports 
  FROM city_reports, per_state AS state
  WHERE state.State = city_reports.State
  AND city_reports.Reports > 10
'''
ca_cities = ps.sqldf(query, locals())

,City,Shape,Reports
0,Phoenix,Flash,558
1,Seattle,Changing,548
2,Portland,Circle,480
3,Las Vegas,Rectangle,473
4,San Diego,Rectangle,394
...,...,...,...
1654,Winnemucca,Cylinder,11
1655,Wrightsville Beach,Other,11
1656,Xenia,Other,11
1657,Yorktown,Rectangle,11


In [39]:
query = '''
  SELECT city_reports.City, city_reports.Shape, city_reports.Reports 
  FROM city_reports, most_reported_state AS mr_state
  WHERE mr_state.State = city_reports.State
  AND city_reports.Reports > 10
  AND city_reports.Shape !='Unknown'
  AND city_reports.Shape !=''
  '''
ca_cities = ps.sqldf(query, locals())

,City,Shape,Reports
0,San Diego,Rectangle,394
1,Los Angeles,Changing,379
2,Sacramento,Changing,242
3,San Jose,Other,223
4,San Francisco,Chevron,200
...,...,...,...
221,Imperial Beach,Fireball,11
222,Miramar,Formation,11
223,Nipomo,Fireball,11
224,Pismo Beach,Disk,11
